In [1]:
import pandas as pd
import seaborn as sns

%load_ext google.cloud.bigquery

## Base Tables

In [2]:
params = {
    'report_start_date': '2023-01-01',
    'report_end_date': '2023-01-31'
}

In [7]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Unsubs_v3_Jan23` AS (
    SELECT DISTINCT adobe_tracking_id
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Unsubscribes' 
    AND event_date <= @report_end_date
)

Query is running:   0%|          |

""


In [8]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Targeted_v3_Jan23` AS ( -- everyone who have received emails in the month, and opened at least one email in the past
    SELECT DISTINCT adobe_tracking_id 
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Deliveries' 
    AND event_date BETWEEN @report_start_date and @report_end_date
)

Query is running:   0%|          |

""


In [9]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Holdout_v3_Jan23` AS ( 
    SELECT  distinct holdout.adobe_tracking_id
    FROM
    (
        SELECT  distinct TrackingId AS adobe_tracking_id -- distinct Hold_Out_Type_Current --cohort, count(distinct TrackingId)
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER)) -- get cohort name as month of quarter start + year
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
        -- Exclude those who are assigned to Email Holdout but actually received emails 
    ) holdout
    LEFT JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
        WHERE event_name = 'Email Deliveries'
        AND event_date BETWEEN @report_start_date AND @report_end_date
        AND lower(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails
    ) delivered -- Email Delivered ever
    ON holdout.adobe_tracking_id = delivered.adobe_tracking_id AND delivered.adobe_tracking_id is null
)

Query is running:   0%|          |

""


### Output Base Tables

In [26]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_v3_10M_test_Jan23` AS (
    -- email targetable and holdout cohort, users signed up after 2020/8/11
     SELECT distinct a.aid
     , a.cohort
     , a.account_type
     , acc_type_start.account_type as account_type_start
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_v3_10M_Jan23` a
    --add attribute: account_type at the start of the month 
    LEFT JOIN ( SELECT * FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` WHERE report_date = @report_start_date ) acc_type_start
    ON a.aid = acc_type_start.adobe_tracking_id 
)

Query is running:   0%|          |

""


In [9]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Video_Viewing_v3_10M_Jan23` AS (
    SELECT 
        adobe_tracking_id
        , COUNT (DISTINCT CASE WHEN VIDEO.num_views_started = 1 THEN video.adobe_tracking_id ELSE NULL END) AS Distinct_Content_Starts
        , SUM (VIDEO.num_views_started ) AS Total_Content_Starts
        , SUM(VIDEO.num_seconds_played_no_ads)/3600 AS Viewing_Time
        , COUNT(DISTINCT session_id) AS Distinct_Viewing_Sessions 
        , COUNT(DISTINCT(CASE WHEN (num_seconds_played_no_ads > CASE WHEN lower(consumption_type) = 'virtual channel' THEN 299 ELSE 0 END)
                and (num_views_started>0) THEN CASE WHEN (lower(consumption_type) = "shortform") THEN "Shortform"
                                                    WHEN lower(franchise) != 'other' THEN franchise ELSE display_name
                                                    END
                end)) as Repertoire_Pavo_Method
                
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_v3_10M_Jan23` a

    INNER JOIN  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO
        ON VIDEO.adobe_tracking_id = a.aid
        AND adobe_date between @report_start_date AND @report_end_date
    GROUP BY 1
)

Query is running:   0%|          |

""


## Metric Definitions

In [28]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v3_10M_test_Jan23` AS 

WITH
Save_Denom AS ( --'Lapsing_Users'
    SELECT distinct adobe_tracking_id, date_of_last_view -- no need for max (days_since_last_view), can be saved multiple times, dedup later
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
    WHERE report_date BETWEEN @report_start_date AND @report_end_date 
    AND days_since_last_view BETWEEN 15 AND 29 -- this guarantees we are only getting people who have at least past the 'lapsing' phase in the time period.
)
, Save_Num as (
        SELECT distinct a.adobe_tracking_id      
        FROM Save_Denom a
        INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO 
            ON a.adobe_tracking_id = VIDEO.adobe_tracking_id 
            AND adobe_date BETWEEN @report_start_date AND @report_end_date
            AND VIDEO.adobe_date BETWEEN DATE_ADD(date_of_last_view, INTERVAL 15 day) AND DATE_ADD(date_of_last_view, INTERVAL 29 day)
)
, Winback_Denom AS ( --'Lapsed_Users'
    SELECT distinct adobe_tracking_id, date_of_last_view
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
    WHERE report_date BETWEEN @report_start_date AND @report_end_date 
    AND days_since_last_view BETWEEN 30 AND 90-- this guarantees we are only getting people who have at least past the 'lapsing' phase in the time period.
    )
-- REVIEW: compare this code to email dash (Gold table)
, Winback_Num AS (
    SELECT distinct a.adobe_tracking_id
    FROM Winback_Denom a
    INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO`  VIDEO 
        ON a.adobe_tracking_id = VIDEO.adobe_tracking_id 
        AND adobe_date BETWEEN @report_start_date AND @report_end_date 
        AND VIDEO.adobe_date BETWEEN DATE_ADD(date_of_last_view, INTERVAL 30 day) 
            AND DATE_ADD(date_of_last_view, INTERVAL 90 DAY)
    )
, Upgrade_Denom AS (
-- upgrade metric 2.0: nonpaying to paying
    SELECT distinct adobe_tracking_id 
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`  USER
    WHERE paying_account_flag = 'NonPaying' 
    AND USER.report_date BETWEEN @report_start_date and @report_end_date
)
, Upgrade_Num AS (
-- find all users who have upgraded at least once in the month of
    SELECT  distinct adobe_tracking_id
    FROM
        (SELECT  
                report_date
             , adobe_tracking_id     
         FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`  USER
         WHERE entitlement_change_flag IN ('Upgrade: Free to Premium'
                                         , 'Upgrade: Free to Premium+'
                                         , 'Upgrade: Premium to Premium+') 
         AND paying_account_flag = 'Paying'                                            
            AND USER.report_date BETWEEN @report_start_date and @report_end_date
        ) 
)    
, Paid_Churn_Denom AS (
    SELECT distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
    WHERE paying_account_flag = 'Paying'
    AND auto_renew_flag = 'OFF'
    AND report_date BETWEEN @report_start_date and @report_end_date
)
, Paid_Churn_Num AS (
    SELECT  distinct adobe_tracking_id
    FROM
    (
        SELECT  adobe_tracking_id
            ,report_date
            ,auto_renew_flag                                                                      AS auto_renew_flag_today
            ,LEAD(auto_renew_flag,1) OVER ( partition by adobe_tracking_id ORDER BY report_date ) AS auto_renew_flag_next_day
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE report_date BETWEEN @report_start_date AND @report_end_date
        ORDER BY 1, 2 
    )
    WHERE auto_renew_flag_today = 'OFF'
    AND auto_renew_flag_next_day = 'ON'
)
, New_Upgrade_Denom AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_never`
    WHERE report_date BETWEEN @report_start_date AND @report_end_date
)
, New_Upgrade_Num AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_date_rank`
    WHERE upgrade_row_number = 1
    AND report_date BETWEEN @report_start_date AND @report_end_date
)
, Paid_Winbacks_Denom AS (
    SELECT  distinct adobe_tracking_id
    FROM
    (
        SELECT  adobe_tracking_id
            ,report_date
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE paying_account_flag = 'NonPaying'
        AND report_date BETWEEN @report_start_date AND @report_end_date 
    )
    WHERE adobe_tracking_id NOT IN ( SELECT distinct adobe_tracking_id FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_never` WHERE report_date BETWEEN @report_start_date AND @report_end_date)
)
, Paid_Winbacks_Num AS (
    select distinct adobe_tracking_id
    from `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_date_rank`
    where upgrade_row_number > 1
    and report_date between @report_start_date and @report_end_date
)
-- this metric uses silver_churn, which is based on data in PAVO dash Churn Trend
, EOM_Paid_Churn_Denom AS (
  select adobe_tracking_id
  from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
  where date = @report_end_date
  and entitlement = 'Paid'
)
, EOM_Paid_Churn_Num AS (
  select adobe_tracking_id
  from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
  where date = @report_end_date
  and entitlement = 'Paid'
  and Churn_flag = 'Churn'
)
-- Where base date looks at all users at the start of the month
, EOM_Paid_Churn_Denom_report_start AS (
  select adobe_tracking_id
  from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
  where base_date = @report_start_date
  and entitlement = 'Paid'
)
, EOM_Paid_Churn_Num_report_start AS (
  select adobe_tracking_id
  from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
  where base_date = @report_start_date
  and entitlement = 'Paid'
  and Churn_flag = 'Churn'
)
SELECT  @report_start_date                     AS Report_Month
       ,a.Account_Type --EOM 1/31 
       ,a.Account_Type_Start --SOM 1/1
       ,a.aid
       ,a.cohort
       ,video.adobe_tracking_id                AS viewers
       ,video.Viewing_Time
       ,video.Repertoire_Pavo_Method
       ,video.Distinct_Viewing_Sessions
       ,Winback_Denom.adobe_tracking_id        AS Winback_Denom
       ,Winback_Num.adobe_tracking_id          AS Winback_Num
       ,Save_Denom.adobe_tracking_id           AS Save_Denom
       ,Save_Num.adobe_tracking_id             AS Save_Num
       ,Upgrade_Denom.adobe_tracking_id        AS Upgrade_Denom
       ,Upgrade_Num.adobe_tracking_id          AS Upgrade_Num
       ,New_Upgrade_Denom.adobe_tracking_id    AS New_Upgrade_Denom
       ,New_Upgrade_Num.adobe_tracking_id      AS New_Upgrade_Num
       ,Paid_Winbacks_Denom.adobe_tracking_id  AS Paid_Winbacks_Denom
       ,Paid_Winbacks_Num.adobe_tracking_id    AS Paid_Winbacks_Num
       ,Paid_Churn_Denom.adobe_tracking_id     AS Paid_Churn_Denom
       ,Paid_Churn_Num.adobe_tracking_id       AS Paid_Churn_Num
       -- Churn of base date = 12/31
       ,EOM_Paid_Churn_Denom.adobe_tracking_id AS EOM_Paid_Churn_Denom
       ,EOM_Paid_Churn_Num.adobe_tracking_id   AS EOM_Paid_Churn_Num
       -- Churn of base date = 1/1
       ,EOM_Paid_Churn_Denom_report_start.adobe_tracking_id AS EOM_Paid_Churn_Denom_report_start
       ,EOM_Paid_Churn_Num_report_start.adobe_tracking_id   AS EOM_Paid_Churn_Num_report_start
FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_v3_10M_test_Jan23` a
LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Video_Viewing_v3_10M_Jan23` video
    ON a.aid = video.adobe_tracking_id
LEFT JOIN Winback_Denom
    ON a.aid = Winback_Denom.adobe_tracking_id
LEFT JOIN Winback_Num
    ON Winback_Denom.adobe_tracking_id = Winback_Num.adobe_tracking_id
LEFT JOIN Save_Denom
    ON a.aid = Save_Denom.adobe_tracking_id
LEFT JOIN Save_Num
    ON Save_Denom.adobe_tracking_id = Save_Num.adobe_tracking_id
LEFT JOIN Upgrade_Denom
    ON a.aid = Upgrade_Denom.adobe_tracking_id
LEFT JOIN Upgrade_Num 
    ON Upgrade_Denom.adobe_tracking_id = Upgrade_Num.adobe_tracking_id
LEFT JOIN New_Upgrade_Denom
    on a.aid = New_Upgrade_Denom.adobe_tracking_id
LEFT JOIN New_Upgrade_Num
    on New_Upgrade_Denom.adobe_tracking_id = New_Upgrade_Num.adobe_tracking_id
LEFT JOIN Paid_Winbacks_Denom
    on a.aid = Paid_Winbacks_Denom.adobe_tracking_id
LEFT JOIN Paid_Winbacks_Num
    on Paid_Winbacks_Num.adobe_tracking_id = Paid_Winbacks_Denom.adobe_tracking_id
LEFT JOIN Paid_Churn_Denom
    on a.aid = Paid_Churn_Denom.adobe_tracking_id
LEFT JOIN Paid_Churn_Num
    on Paid_Churn_Denom.adobe_tracking_id = Paid_Churn_Num.adobe_tracking_id
LEFT JOIN EOM_Paid_Churn_Denom
    on a.aid = EOM_Paid_Churn_Denom.adobe_tracking_id
LEFT JOIN EOM_Paid_Churn_Num
    on EOM_Paid_Churn_Denom.adobe_tracking_id = EOM_Paid_Churn_Num.adobe_tracking_id

LEFT JOIN EOM_Paid_Churn_Denom_report_start
    on a.aid = EOM_Paid_Churn_Denom_report_start.adobe_tracking_id
LEFT JOIN EOM_Paid_Churn_Num_report_start
    on EOM_Paid_Churn_Denom_report_start.adobe_tracking_id = EOM_Paid_Churn_Num_report_start.adobe_tracking_id

Query is running:   0%|          |

""


# Analysis

In [13]:
%%bigquery df_tot --project nbcu-ds-sandbox-a-001 --params $params

WITH 
CTE_1 AS (
  SELECT 
    @report_start_date AS Report_Month
    --, Account_Type
    , count(distinct case when cohort = 'Email_Targeted' then aid end ) as Distinct_Cohort_Size_Targeted
    , count(distinct case when cohort = 'Holdout' then aid end) as Distinct_Cohort_Size_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then viewers end ) as Total_Returns_Targeted
    , count(distinct case when cohort = 'Holdout' then viewers end) as Total_Returns_Holdout

    , sum(case when cohort = 'Email_Targeted' then Viewing_Time end ) as Total_Usage_Targeted
    , sum(case when cohort = 'Holdout' then Viewing_Time end) as Total_Usage_Holdout

    , sum(case when cohort = 'Email_Targeted' then Repertoire_Pavo_Method end ) as Total_Repertoire_Targeted
    , sum(case when cohort = 'Holdout' then Repertoire_Pavo_Method end) as Total_Repertoire_Holdout

    , sum(case when cohort = 'Email_Targeted' then Distinct_Viewing_Sessions end ) as Total_Viewing_Sessions_Targeted
    , sum(case when cohort = 'Holdout' then Distinct_Viewing_Sessions end) as Total_Viewing_Sessions_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Denom end ) as EOM_Paid_Churn_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Num end) as EOM_Paid_Churn_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Denom end ) as EOM_Paid_Churn_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Num end) as EOM_Paid_Churn_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Denom_report_start end ) as EOM_Paid_Churn_Denom_Targeted_report_start
    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Num_report_start end) as EOM_Paid_Churn_Num_Targeted_report_start
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Denom_report_start end ) as EOM_Paid_Churn_Denom_Holdout_report_start
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Num_report_start end) as EOM_Paid_Churn_Num_Holdout_report_start
  FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v3_10M_test2_Jan23`
  GROUP BY 1 --,2
), CTE_2 AS (
    SELECT Report_Month
  --, Account_Type 
  , Distinct_Cohort_Size_Targeted   as Email_Engagers
  , Distinct_Cohort_Size_Holdout   as Email_Holdout


  , safe_divide(Total_Returns_Targeted, Distinct_Cohort_Size_Targeted)  as Return_Rate_Engagers
  , safe_divide(Total_Returns_Holdout, Distinct_Cohort_Size_Holdout)  as Return_Rate_Holdout


  , safe_divide(Total_Usage_Targeted, Distinct_Cohort_Size_Targeted)  as Usage_Engagers
  , safe_divide(Total_Usage_Holdout, Distinct_Cohort_Size_Holdout)   as Usage_Holdout

  , safe_divide(Total_Repertoire_Targeted, Distinct_Cohort_Size_Targeted)  as Repertoire_Engagers
  , safe_divide(Total_Repertoire_Holdout,Distinct_Cohort_Size_Holdout)   as Repertoire_Holdout


  , safe_divide(Total_Viewing_Sessions_Targeted, Distinct_Cohort_Size_Targeted)  as Sessions_Engagers
  , safe_divide(Total_Viewing_Sessions_Holdout, Distinct_Cohort_Size_Holdout)   as Sessions_Holdout


  , EOM_Paid_Churn_Denom_Targeted
  , safe_divide(EOM_Paid_Churn_Num_Targeted, EOM_Paid_Churn_Denom_Targeted) as Paid_Churn_Rate_Engagers
  , safe_divide(EOM_Paid_Churn_Num_Holdout, EOM_Paid_Churn_Denom_Holdout) as Paid_Churn_Rate_Holdout

  , EOM_Paid_Churn_Denom_Targeted_report_start
  , safe_divide(EOM_Paid_Churn_Num_Targeted_report_start, EOM_Paid_Churn_Denom_Targeted_report_start) as Paid_Churn_Rate_Engagers_report_start
  , safe_divide(EOM_Paid_Churn_Num_Holdout_report_start, EOM_Paid_Churn_Denom_Holdout_report_start) as Paid_Churn_Rate_Holdout_report_start

  from CTE_1
)

SELECT Report_Month
--, Account_Type
, Email_Engagers
, Email_Holdout


, Return_Rate_Engagers                                                   as Return_Rate_Engagers
, Return_Rate_Holdout                                                    as Return_Rate_Holdout
, Return_Rate_Engagers - Return_Rate_Holdout                             as Return_Rate_Lift_PTS
, safe_divide(Return_Rate_Engagers, Return_Rate_Holdout) *100                        as Return_Rate_Lift_Index
, (Return_Rate_Engagers - Return_Rate_Holdout) * Email_Engagers          as Returns_Incrementals

, Usage_Engagers                                                         as Usage_Engagers
, Usage_Holdout                                                          as Usage_Holdout
, Usage_Engagers - Usage_Holdout                                         as Usage_Lift_PTS
, safe_divide(Usage_Engagers, Usage_Holdout) *100                                    as Usage_Lift_Index
, (Usage_Engagers - Usage_Holdout) * Email_Engagers                      as Usage_Incrementals


, Repertoire_Engagers                                                    as Repertoire_Engagers
, Repertoire_Holdout                                                     as Repertoire_Holdout
, Repertoire_Engagers - Repertoire_Holdout                               as Repertoire_Lift_PTS
, safe_divide(Repertoire_Engagers, Repertoire_Holdout) *100                          as Repertoire_Lift_Index
, (Repertoire_Engagers - Repertoire_Holdout) * Email_Engagers as Repertoire_Incrementals


, Sessions_Engagers                                                      as Sessions_Engagers
, Sessions_Holdout                                                       as Sessions_Holdout
, Sessions_Engagers - Sessions_Holdout                                   as Sessions_Lift_PTS
, safe_divide(Sessions_Engagers, Sessions_Holdout) *100                              as Sessions_Lift_Index
, (Sessions_Engagers - Sessions_Holdout) * Email_Engagers as Sessions_Incrementals

, Paid_Churn_Rate_Engagers                                                  as EOM_Paid_Churn_Rate_Engagers
, Paid_Churn_Rate_Holdout                                                   as EOM_Paid_Churn_Rate_Holdout
, Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                        as EOM_Paid_Churn_Rate_Lift_PTS
, safe_divide(Paid_Churn_Rate_Engagers, Paid_Churn_Rate_Holdout) *100                   as EOM_Paid_Churn_Rate_Lift_Index
, (Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted as EOM_Paid_Churn_Rate_Incrementals

, Paid_Churn_Rate_Engagers_report_start                                                  as EOM_Paid_Churn_Rate_Engagers_report_start
, Paid_Churn_Rate_Holdout_report_start                                                   as EOM_Paid_Churn_Rate_Holdout_report_start
, Paid_Churn_Rate_Engagers_report_start - Paid_Churn_Rate_Holdout_report_start                        as EOM_Paid_Churn_Rate_Lift_PTS_report_start
, safe_divide(Paid_Churn_Rate_Engagers_report_start, Paid_Churn_Rate_Holdout_report_start) *100                   as EOM_Paid_Churn_Rate_Lift_Index_report_start
, (Paid_Churn_Rate_Engagers_report_start - Paid_Churn_Rate_Holdout_report_start) * EOM_Paid_Churn_Denom_Targeted_report_start as EOM_Paid_Churn_Rate_Incrementals_report_start

FROM CTE_2

Query is running:   0%|          |

Downloading:   0%|          |

In [14]:
df_tot

,Report_Month,Email_Engagers,Email_Holdout,Return_Rate_Engagers,Return_Rate_Holdout,Return_Rate_Lift_PTS,Return_Rate_Lift_Index,Returns_Incrementals,Usage_Engagers,Usage_Holdout,...,EOM_Paid_Churn_Rate_Engagers,EOM_Paid_Churn_Rate_Holdout,EOM_Paid_Churn_Rate_Lift_PTS,EOM_Paid_Churn_Rate_Lift_Index,EOM_Paid_Churn_Rate_Incrementals,EOM_Paid_Churn_Rate_Engagers_report_start,EOM_Paid_Churn_Rate_Holdout_report_start,EOM_Paid_Churn_Rate_Lift_PTS_report_start,EOM_Paid_Churn_Rate_Lift_Index_report_start,EOM_Paid_Churn_Rate_Incrementals_report_start
0,2023-01-01,18958522,843409,0.454597,0.391492,0.063105,116.119003,1.196371e+06,8.532282,7.337942,...,0.066831,0.058874,0.007957,113.514895,45149.184291,0.067,0.05894,0.00806,113.675301,45894.634115


In [8]:
%%bigquery df_acc --project nbcu-ds-sandbox-a-001 --params $params

WITH 
CTE_1 AS (
  SELECT 
    @report_start_date AS Report_Month
    , Account_Type
    , count(distinct case when cohort = 'Email_Targeted' then aid end ) as Distinct_Cohort_Size_Targeted
    , count(distinct case when cohort = 'Holdout' then aid end) as Distinct_Cohort_Size_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then viewers end ) as Total_Returns_Targeted
    , count(distinct case when cohort = 'Holdout' then viewers end) as Total_Returns_Holdout

    , sum(case when cohort = 'Email_Targeted' then Viewing_Time end ) as Total_Usage_Targeted
    , sum(case when cohort = 'Holdout' then Viewing_Time end) as Total_Usage_Holdout

    , sum(case when cohort = 'Email_Targeted' then Repertoire_Pavo_Method end ) as Total_Repertoire_Targeted
    , sum(case when cohort = 'Holdout' then Repertoire_Pavo_Method end) as Total_Repertoire_Holdout

    , sum(case when cohort = 'Email_Targeted' then Distinct_Viewing_Sessions end ) as Total_Viewing_Sessions_Targeted
    , sum(case when cohort = 'Holdout' then Distinct_Viewing_Sessions end) as Total_Viewing_Sessions_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Denom end ) as EOM_Paid_Churn_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Num end) as EOM_Paid_Churn_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Denom end ) as EOM_Paid_Churn_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Num end) as EOM_Paid_Churn_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Denom_report_start end ) as EOM_Paid_Churn_Denom_Targeted_report_start
    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Num_report_start end) as EOM_Paid_Churn_Num_Targeted_report_start
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Denom_report_start end ) as EOM_Paid_Churn_Denom_Holdout_report_start
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Num_report_start end) as EOM_Paid_Churn_Num_Holdout_report_start
  FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v3_10M_test2_Jan23`
  GROUP BY 1 ,2
), CTE_2 AS (
    SELECT Report_Month
  , Account_Type 
  , Distinct_Cohort_Size_Targeted   as Email_Engagers
  , Distinct_Cohort_Size_Holdout   as Email_Holdout


  , safe_divide(Total_Returns_Targeted, Distinct_Cohort_Size_Targeted)  as Return_Rate_Engagers
  , safe_divide(Total_Returns_Holdout, Distinct_Cohort_Size_Holdout)  as Return_Rate_Holdout


  , safe_divide(Total_Usage_Targeted, Distinct_Cohort_Size_Targeted)  as Usage_Engagers
  , safe_divide(Total_Usage_Holdout, Distinct_Cohort_Size_Holdout)   as Usage_Holdout

  , safe_divide(Total_Repertoire_Targeted, Distinct_Cohort_Size_Targeted)  as Repertoire_Engagers
  , safe_divide(Total_Repertoire_Holdout,Distinct_Cohort_Size_Holdout)   as Repertoire_Holdout


  , safe_divide(Total_Viewing_Sessions_Targeted, Distinct_Cohort_Size_Targeted)  as Sessions_Engagers
  , safe_divide(Total_Viewing_Sessions_Holdout, Distinct_Cohort_Size_Holdout)   as Sessions_Holdout


  , EOM_Paid_Churn_Denom_Targeted
  , safe_divide(EOM_Paid_Churn_Num_Targeted, EOM_Paid_Churn_Denom_Targeted) as Paid_Churn_Rate_Engagers
  , safe_divide(EOM_Paid_Churn_Num_Holdout, EOM_Paid_Churn_Denom_Holdout) as Paid_Churn_Rate_Holdout

  , EOM_Paid_Churn_Denom_Targeted_report_start
  , safe_divide(EOM_Paid_Churn_Num_Targeted_report_start, EOM_Paid_Churn_Denom_Targeted_report_start) as Paid_Churn_Rate_Engagers_report_start
  , safe_divide(EOM_Paid_Churn_Num_Holdout_report_start, EOM_Paid_Churn_Denom_Holdout_report_start) as Paid_Churn_Rate_Holdout_report_start

  from CTE_1
)

SELECT Report_Month
, Account_Type
, Email_Engagers
, Email_Holdout


, Return_Rate_Engagers                                                   as Return_Rate_Engagers
, Return_Rate_Holdout                                                    as Return_Rate_Holdout
, Return_Rate_Engagers - Return_Rate_Holdout                             as Return_Rate_Lift_PTS
, safe_divide(Return_Rate_Engagers, Return_Rate_Holdout) *100                        as Return_Rate_Lift_Index
, (Return_Rate_Engagers - Return_Rate_Holdout) * Email_Engagers          as Returns_Incrementals

, Usage_Engagers                                                         as Usage_Engagers
, Usage_Holdout                                                          as Usage_Holdout
, Usage_Engagers - Usage_Holdout                                         as Usage_Lift_PTS
, safe_divide(Usage_Engagers, Usage_Holdout) *100                                    as Usage_Lift_Index
, (Usage_Engagers - Usage_Holdout) * Email_Engagers                      as Usage_Incrementals


, Repertoire_Engagers                                                    as Repertoire_Engagers
, Repertoire_Holdout                                                     as Repertoire_Holdout
, Repertoire_Engagers - Repertoire_Holdout                               as Repertoire_Lift_PTS
, safe_divide(Repertoire_Engagers, Repertoire_Holdout) *100                          as Repertoire_Lift_Index
, (Repertoire_Engagers - Repertoire_Holdout) * Email_Engagers as Repertoire_Incrementals


, Sessions_Engagers                                                      as Sessions_Engagers
, Sessions_Holdout                                                       as Sessions_Holdout
, Sessions_Engagers - Sessions_Holdout                                   as Sessions_Lift_PTS
, safe_divide(Sessions_Engagers, Sessions_Holdout) *100                              as Sessions_Lift_Index
, (Sessions_Engagers - Sessions_Holdout) * Email_Engagers as Sessions_Incrementals

, Paid_Churn_Rate_Engagers                                                  as EOM_Paid_Churn_Rate_Engagers
, Paid_Churn_Rate_Holdout                                                   as EOM_Paid_Churn_Rate_Holdout
, Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                        as EOM_Paid_Churn_Rate_Lift_PTS
, safe_divide(Paid_Churn_Rate_Engagers, Paid_Churn_Rate_Holdout) *100                   as EOM_Paid_Churn_Rate_Lift_Index
, (Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted as EOM_Paid_Churn_Rate_Incrementals

, Paid_Churn_Rate_Engagers_report_start                                                  as EOM_Paid_Churn_Rate_Engagers_report_start
, Paid_Churn_Rate_Holdout_report_start                                                   as EOM_Paid_Churn_Rate_Holdout_report_start
, Paid_Churn_Rate_Engagers_report_start - Paid_Churn_Rate_Holdout_report_start                        as EOM_Paid_Churn_Rate_Lift_PTS_report_start
, safe_divide(Paid_Churn_Rate_Engagers_report_start, Paid_Churn_Rate_Holdout_report_start) *100                   as EOM_Paid_Churn_Rate_Lift_Index_report_start
, (Paid_Churn_Rate_Engagers_report_start - Paid_Churn_Rate_Holdout_report_start) * EOM_Paid_Churn_Denom_Targeted_report_start as EOM_Paid_Churn_Rate_Incrementals_report_start

FROM CTE_2

Query is running:   0%|          |

Downloading:   0%|          |

In [9]:
df_acc = df_acc.sort_values(by='Account_Type')
df_acc

,Report_Month,Account_Type,Email_Engagers,Email_Holdout,Return_Rate_Engagers,Return_Rate_Holdout,Return_Rate_Lift_PTS,Return_Rate_Lift_Index,Returns_Incrementals,Usage_Engagers,...,EOM_Paid_Churn_Rate_Engagers,EOM_Paid_Churn_Rate_Holdout,EOM_Paid_Churn_Rate_Lift_PTS,EOM_Paid_Churn_Rate_Lift_Index,EOM_Paid_Churn_Rate_Incrementals,EOM_Paid_Churn_Rate_Engagers_report_start,EOM_Paid_Churn_Rate_Holdout_report_start,EOM_Paid_Churn_Rate_Lift_PTS_report_start,EOM_Paid_Churn_Rate_Lift_Index_report_start,EOM_Paid_Churn_Rate_Incrementals_report_start
2,2023-01-01,Bundled Premium,3788739,159240,0.569666,0.540561,0.029104,105.384101,110268.878542,8.580380,...,1.0,1.0,0.0,100.0,0.0,0.997917,0.992095,0.005822,100.586819,41.916996
0,2023-01-01,Free,9452051,470277,0.176954,0.141355,0.035599,125.184029,336482.438123,0.585700,...,1.0,1.0,0.0,100.0,0.0,0.993140,0.993516,-0.000376,99.962127,-136.851870
1,2023-01-01,Paying SVOD,5717732,213892,0.837323,0.830480,0.006844,100.824050,39129.681409,21.637002,...,0.0,0.0,0.0,NaN,0.0,0.002462,0.002128,0.000334,115.710462,1779.724431


In [20]:
%%bigquery df_acc2 --project nbcu-ds-sandbox-a-001 --params $params

WITH 
CTE_1 AS (
  SELECT 
    @report_start_date AS Report_Month
    , Account_Type_Start
    , count(distinct case when cohort = 'Email_Targeted' then aid end ) as Distinct_Cohort_Size_Targeted
    , count(distinct case when cohort = 'Holdout' then aid end) as Distinct_Cohort_Size_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then viewers end ) as Total_Returns_Targeted
    , count(distinct case when cohort = 'Holdout' then viewers end) as Total_Returns_Holdout

    , sum(case when cohort = 'Email_Targeted' then Viewing_Time end ) as Total_Usage_Targeted
    , sum(case when cohort = 'Holdout' then Viewing_Time end) as Total_Usage_Holdout

    , sum(case when cohort = 'Email_Targeted' then Repertoire_Pavo_Method end ) as Total_Repertoire_Targeted
    , sum(case when cohort = 'Holdout' then Repertoire_Pavo_Method end) as Total_Repertoire_Holdout

    , sum(case when cohort = 'Email_Targeted' then Distinct_Viewing_Sessions end ) as Total_Viewing_Sessions_Targeted
    , sum(case when cohort = 'Holdout' then Distinct_Viewing_Sessions end) as Total_Viewing_Sessions_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Denom end ) as EOM_Paid_Churn_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Num end) as EOM_Paid_Churn_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Denom end ) as EOM_Paid_Churn_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Num end) as EOM_Paid_Churn_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Denom_report_start end ) as EOM_Paid_Churn_Denom_Targeted_report_start
    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Num_report_start end) as EOM_Paid_Churn_Num_Targeted_report_start
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Denom_report_start end ) as EOM_Paid_Churn_Denom_Holdout_report_start
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Num_report_start end) as EOM_Paid_Churn_Num_Holdout_report_start
  FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v3_10M_test2_Jan23`
  GROUP BY 1 ,2
), CTE_2 AS (
    SELECT Report_Month
  , Account_Type_Start
  , Distinct_Cohort_Size_Targeted   as Email_Engagers
  , Distinct_Cohort_Size_Holdout   as Email_Holdout


  , safe_divide(Total_Returns_Targeted, Distinct_Cohort_Size_Targeted)  as Return_Rate_Engagers
  , safe_divide(Total_Returns_Holdout, Distinct_Cohort_Size_Holdout)  as Return_Rate_Holdout


  , safe_divide(Total_Usage_Targeted, Distinct_Cohort_Size_Targeted)  as Usage_Engagers
  , safe_divide(Total_Usage_Holdout, Distinct_Cohort_Size_Holdout)   as Usage_Holdout

  , safe_divide(Total_Repertoire_Targeted, Distinct_Cohort_Size_Targeted)  as Repertoire_Engagers
  , safe_divide(Total_Repertoire_Holdout,Distinct_Cohort_Size_Holdout)   as Repertoire_Holdout


  , safe_divide(Total_Viewing_Sessions_Targeted, Distinct_Cohort_Size_Targeted)  as Sessions_Engagers
  , safe_divide(Total_Viewing_Sessions_Holdout, Distinct_Cohort_Size_Holdout)   as Sessions_Holdout


  , EOM_Paid_Churn_Denom_Targeted
  , safe_divide(EOM_Paid_Churn_Num_Targeted, EOM_Paid_Churn_Denom_Targeted) as Paid_Churn_Rate_Engagers
  , safe_divide(EOM_Paid_Churn_Num_Holdout, EOM_Paid_Churn_Denom_Holdout) as Paid_Churn_Rate_Holdout

  , EOM_Paid_Churn_Denom_Targeted_report_start
  , safe_divide(EOM_Paid_Churn_Num_Targeted_report_start, EOM_Paid_Churn_Denom_Targeted_report_start) as Paid_Churn_Rate_Engagers_report_start
  , safe_divide(EOM_Paid_Churn_Num_Holdout_report_start, EOM_Paid_Churn_Denom_Holdout_report_start) as Paid_Churn_Rate_Holdout_report_start

  from CTE_1
)

SELECT Report_Month
, Account_Type_Start
, Email_Engagers
, Email_Holdout


, Return_Rate_Engagers                                                   as Return_Rate_Engagers
, Return_Rate_Holdout                                                    as Return_Rate_Holdout
, Return_Rate_Engagers - Return_Rate_Holdout                             as Return_Rate_Lift_PTS
, safe_divide(Return_Rate_Engagers, Return_Rate_Holdout) *100                        as Return_Rate_Lift_Index
, (Return_Rate_Engagers - Return_Rate_Holdout) * Email_Engagers          as Returns_Incrementals

, Usage_Engagers                                                         as Usage_Engagers
, Usage_Holdout                                                          as Usage_Holdout
, Usage_Engagers - Usage_Holdout                                         as Usage_Lift_PTS
, safe_divide(Usage_Engagers, Usage_Holdout) *100                                    as Usage_Lift_Index
, (Usage_Engagers - Usage_Holdout) * Email_Engagers                      as Usage_Incrementals


, Repertoire_Engagers                                                    as Repertoire_Engagers
, Repertoire_Holdout                                                     as Repertoire_Holdout
, Repertoire_Engagers - Repertoire_Holdout                               as Repertoire_Lift_PTS
, safe_divide(Repertoire_Engagers, Repertoire_Holdout) *100                          as Repertoire_Lift_Index
, (Repertoire_Engagers - Repertoire_Holdout) * Email_Engagers as Repertoire_Incrementals


, Sessions_Engagers                                                      as Sessions_Engagers
, Sessions_Holdout                                                       as Sessions_Holdout
, Sessions_Engagers - Sessions_Holdout                                   as Sessions_Lift_PTS
, safe_divide(Sessions_Engagers, Sessions_Holdout) *100                              as Sessions_Lift_Index
, (Sessions_Engagers - Sessions_Holdout) * Email_Engagers as Sessions_Incrementals

, Paid_Churn_Rate_Engagers                                                  as EOM_Paid_Churn_Rate_Engagers
, Paid_Churn_Rate_Holdout                                                   as EOM_Paid_Churn_Rate_Holdout
, Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                        as EOM_Paid_Churn_Rate_Lift_PTS
, safe_divide(Paid_Churn_Rate_Engagers, Paid_Churn_Rate_Holdout) *100                   as EOM_Paid_Churn_Rate_Lift_Index
, (Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted as EOM_Paid_Churn_Rate_Incrementals

, Paid_Churn_Rate_Engagers_report_start                                                  as EOM_Paid_Churn_Rate_Engagers_report_start
, Paid_Churn_Rate_Holdout_report_start                                                   as EOM_Paid_Churn_Rate_Holdout_report_start
, Paid_Churn_Rate_Engagers_report_start - Paid_Churn_Rate_Holdout_report_start                        as EOM_Paid_Churn_Rate_Lift_PTS_report_start
, safe_divide(Paid_Churn_Rate_Engagers_report_start, Paid_Churn_Rate_Holdout_report_start) *100                   as EOM_Paid_Churn_Rate_Lift_Index_report_start
, (Paid_Churn_Rate_Engagers_report_start - Paid_Churn_Rate_Holdout_report_start) * EOM_Paid_Churn_Denom_Targeted_report_start as EOM_Paid_Churn_Rate_Incrementals_report_start

FROM CTE_2

Query is running:   0%|          |

Downloading:   0%|          |

In [22]:
df_acc2 = df_acc2.sort_values(by='Account_Type_Start')
df_acc2

,Report_Month,Account_Type_Start,Email_Engagers,Email_Holdout,Return_Rate_Engagers,Return_Rate_Holdout,Return_Rate_Lift_PTS,Return_Rate_Lift_Index,Returns_Incrementals,Usage_Engagers,...,EOM_Paid_Churn_Rate_Engagers,EOM_Paid_Churn_Rate_Holdout,EOM_Paid_Churn_Rate_Lift_PTS,EOM_Paid_Churn_Rate_Lift_Index,EOM_Paid_Churn_Rate_Incrementals,EOM_Paid_Churn_Rate_Engagers_report_start,EOM_Paid_Churn_Rate_Holdout_report_start,EOM_Paid_Churn_Rate_Lift_PTS_report_start,EOM_Paid_Churn_Rate_Lift_Index_report_start,EOM_Paid_Churn_Rate_Incrementals_report_start
3,2023-01-01,Bundled Premium,4106828,169998,0.564743,0.538595,0.026149,104.855011,107388.732291,8.354471,...,0.904523,1.000000,-0.095477,90.452261,-19.000000,NaN,NaN,NaN,NaN,NaN
2,2023-01-01,Free,9041471,458812,0.172852,0.141278,0.031574,122.349131,285478.882523,0.830901,...,0.779334,0.746959,0.032375,104.334270,342.141119,NaN,NaN,NaN,NaN,NaN
0,2023-01-01,Paying SVOD,5694003,214338,0.811401,0.809688,0.001713,100.211515,9751.616228,20.695898,...,0.065472,0.057510,0.007962,113.843775,45090.855534,0.067,0.05894,0.00806,113.675301,45894.634115
1,2023-01-01,None,116220,261,1.000000,1.000000,0.000000,100.000000,0.000000,18.017676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


End